## Data Profiler

Generate data profile reports from a dataiku dataset or sql query for serious exploratory data analysis. This function uses [pandas_profiling](https://github.com/pandas-profiling/pandas-profiling) to produce data profiles for quick data analysis.

### Generating a data profile:

1. Change python environment (kernel) to one that has the pandas-profiling package installed. You can do this by clicking the kernel menu option from inside the notebook. At the top of the notebook click 'Kernel' > 'Change kernel' > 'Python (env Data-Profiling)
2. Enter paramaters in the pre-built function for data source (SQL vs dataiku dataset) and output location (in the notebook vs on the shared network drive)
3. Run all the cells in the notebook

In [98]:
def data_report(data = None, dss_project = None, SQL = False, 
                sql_connection = "Oracle_Pophealth_Analytics"
                , download = False, userID = None , password = None
               ):
    """Build parameters to create a data report using Pandas-Profiling"""
    import dataiku
    from dataiku import pandasutils as pdu
    import pandas as pd
    from pandas_profiling import ProfileReport
    
    if SQL:
        print("Using SQL Table for report")
        from dataiku.core.sql import SQLExecutor2
        if data is None:
            sql_table = 'Dual'
        else: 
            sql_table = data
        sql_query = ("select * from " + sql_table)
        df = SQLExecutor2(sql_connection).query_to_df(sql_query) # load sql query results into dataframe
    else:
        if dss_project is None:
            import os
            dss_project = os.environ['DKU_CURRENT_PROJECT_KEY'] 
        if data is None:
            import numpy as np
            df = pd.DataFrame(
                np.random.rand(100, 5),
                columns=["a", "b", "c", "d", "e"]
            )
        else: 
            df = dataiku.Dataset(data, dss_project).get_dataframe()
    
    # generate report styling using a custom config
    profile = ProfileReport(df, config_file='/datadrive/data_dir/code-envs/packages/config.yaml')
    
    # if enabled save report to network drive
    if download:
        from smb.SMBConnection import SMBConnection
        import sys
        # set parameters
        if userID is None:
            print("user id is missing, can't connect to network shared drive")
            sys.exit('null userID') 
        if password is None:
            print("password is missing, can't connect to network shared drive")
            sys.exit('null password') 
        client_machine_name = ''
        server_name = 'mshs6290b-cifs'
        server_ip = '10.2.110.53'
        service_name = 'GRPS4'
        target = '/DataikuShare/'
        # create connection to network share
        conn = SMBConnection(userID, password, client_machine_name, server_name)
        
        # test connection
        if conn.connect(server_ip):
            print('connection successful')
            from datetime import datetime as dt
            now=dt.now().strftime('%Y-%m-%d_%H%M')
            local_file = "data_report_" + now + '.html'
            print(' starting to build report at: ' + now)
            local_dir = '/datadrive/data_dir/uploads/'
#             print(' saving report locally to: ' + local_dir)
            file_name = local_dir + local_file
#             print(file_name)
            # create data profile and save to file
            profile.to_file((local_dir + local_file))
            print(' created data report: ' + local_file)
            
            # read report from local file and write to file on network share
            with open((local_dir + local_file), 'rb', buffering=0) as file_obj:
                    print(' saveing ' + local_file + ' to ' + target)
                    conn.storeFile('GRPS4', target + local_file, file_obj)
            if file_obj.closed:
                # if report was written to network drive delete local copy
                import os
                os.remove(file_name)
                conn.close()
                print('connection closed')
        else:
            print('connection error')
    else:
        from IPython.display import HTML
        from datetime import datetime as dt
        now=dt.now().strftime('%Y-%m-%d_%H%M%S')
        local_file = "data_report_" + now + ".html"
        serve_dir = "/datadrive/data_dir/local/static/data_profile/"
        site = "https://dataiku-designer.mountsinai.org/local/static/data_profile/"
#         print(local_file)
        profile.to_file(serve_dir + local_file)
        # def create_download_link( file, title = "Download file", filename = "data_profile"):
#         def create_download_link( title = "Download file", filename = "data_profile"):
        payload= site + local_file
        title = "View Data Profile"
        html = '<a href="{payload}" target="_blank">{title}</a>'
        html = html.format(payload=payload,title=title)
        return HTML(html)
#         return(profile.to_notebook_iframe())
#         return()
        # profile.to_widget()

### Configuring the report

If no parameters are passed, the defualt configuration will create a random dataset and produce the report inside the notebook

#### data
1. To read data in from a dataset in the current dataiku project simply enter the dataset name.
    - `data = 'my_dataset'`
2. To read data in from a dataset in a different dataiku project you will have to specificy the dataset name and the project key.   
    - `data = 'my_dataset_name', dss_project = 'my_project_key'`
    - The project key is provided in the dataiku url immediately after `/project/` (i.e. https://dataiku-designer.mountsinai.org/projects/my_project_key/)
3. To read data in directly from a sql table, enter the table name and set the SQL flag to true.
    - `data = 'my_sql_table', SQL = True` 


#### download

Download the report to the network shared drive by setting the download flag to true.
Since the network drive is not configured in dataiku this program creates the connection. You therefore have to provide your userID and password.
- `download = True, userID='zywota01', password='my_$ecret_pwd'`

In [99]:
# Examples:

# To run the report with default settings uncomment the line below. This will generate a random dataset and display the report in the notebook.
#data_report()

# To run the report with a dataset from the current dataiku project, uncomment the line below. Make sure to enter the name of the dataset you want to use. This will display the report in the notebook. 
#data_report(data='my_dataset')  

# To run the report on a SQL table uncomment the line below. This will run `select * from 'my_table'` and display the report in the notebook. 
#data_report(data='my_table', SQL = True)

# To run the report with default settings and save it to the network shared drive uncomment the line below. You will have to enter you network userID and password to connect to the network drive. This will generate a random dataset and save the report on the network shared drive in the folder DataikuShare.
#data_report( download=True, userID='zywota01', password='my_$ecret_pwd')

In [103]:
data_report(data = 'EPIC_VIDEO_VIS_DET_VW', SQL=True)

Using SQL Table for report
data_report_2020-07-02_1806.html


In [ ]:
EPIC_VIDEO_VIS_AUDIT_VW
EPIC_VIDEO_VIS_DET_VW
V
